# Analysis of Features by Classification Accuracy

In [1]:
import init
import common.constants as cn
from common.trinary_data import TrinaryData
from common.data_provider import DataProvider
import classifier.main_multi_classifier_feature_optimizer as main
from common.data_provider import DataProvider
from common_python.plots import util_plots
from plots import util_plots as xutil_plots

import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

In [2]:
PROVIDER = DataProvider()
PROVIDER.do()
TRINARY = TrinaryData(is_averaged=False, is_dropT1=False)  # Trinary data
DF_CSV = main.makeFitResultCSV(path=None)  # Dataframe of CSV files
FIT_RESULT_DCT = main.getFitResultFromPersister()  # Results dictionary

In [3]:
PROVIDER.df_trn_signed.head()

,tf,GENE_ID,sign
0,Rv0022c,Rv0029,1
1,Rv0022c,Rv0030,1
2,Rv0022c,Rv0031,1
3,Rv0022c,Rv0069c,1
4,Rv0022c,Rv0070c,1


In [4]:

def run():
    for cl in FIT_RESULT_DCT.keys():
        length_stg = str([len(fr.sels) for fr in FIT_RESULT_DCT[cl]])
        score_stg = str([fr.sels_score for fr in FIT_RESULT_DCT[cl]])
        n_eval_stg = str([fr.n_eval for fr in FIT_RESULT_DCT[cl]])
        print("%d: %s\n   %s\n   %s" % (cl, length_stg, score_stg, n_eval_stg))

In [5]:
#run()

In [6]:
# Check status of run
df_g = pd.DataFrame(DF_CSV.groupby([cn.STATE, cn.INDEX]).count())
dff = df_g.reset_index()
for state in DF_CSV[cn.STATE].unique():
    dff_state = dff[dff[cn.STATE] == state]
    print("%d: %d" % (state, dff_state[cn.INDEX].max()))

0: 29
1: 29
2: 29
3: 28
4: 28
5: 29


## Analyze Results By Iteration

In [7]:
# Group by state and index
dfg = DF_CSV.groupby([cn.STATE, cn.INDEX])
df_g = pd.DataFrame(dfg.count())
df_g[cn.COUNT] = dfg.count()
for col in df_g.columns:
    if col != cn.COUNT:
        del df_g[col]              
df_g = df_g.reset_index()
df_g[cn.SCORE] = dfg.mean().reset_index()[cn.SCORE]
state = 0
df = df_g[df_g[cn.STATE] == state]
#plt.scatter(df[cn.INDEX], df[cn.SCORE])
df_g.head()

,state,index,count,score
0,0,0,1,1.000000
1,0,1,1,1.000000
2,0,2,3,1.000000
3,0,3,2,0.993333
4,0,4,2,1.000000


In [8]:
DF_CSV.head()

,state,index,GENE_ID,score,count
0,0,0,Rv2626c,1.0,1
1,0,1,Rv1813c,1.0,1
2,0,2,Rv0022c,1.0,117
3,0,2,Rv2007c,1.0,117
4,0,2,Rv3854c,1.0,117


In [9]:
PROVIDER.df_trn_unsigned

,tf,GENE_ID,sign
0,Rv0022c,Rv0029,1
1,Rv0022c,Rv0030,1
2,Rv0022c,Rv0031,1
3,Rv0022c,Rv0069c,1
4,Rv0022c,Rv0070c,1
...,...,...,...
12183,Rv3862c,Rv3881c,1
12184,Rv3911,Rv2514c,1
12185,Rv3911,Rv2515c,1
12186,Rv3911,Rv3906c,1


In [10]:
df_merge = DF_CSV.merge(PROVIDER.df_trn_unsigned)

In [11]:
df_merge.head()

,state,index,GENE_ID,score,count,tf,sign
0,0,0,Rv2626c,1.0,1,Rv1985c,1
1,0,0,Rv2626c,1.0,1,Rv3133c,1
2,0,1,Rv1813c,1.0,1,Rv1985c,1
3,0,2,Rv0022c,1.0,117,Rv1033c,1
4,0,2,Rv0022c,1.0,117,Rv3597c,1


In [12]:
df_merge = DF_CSV.merge(PROVIDER.df_trn_unsigned)
del df_merge[cn.SIGN]
del df_merge[cn.COUNT]
df_merge = df_merge.drop_duplicates()
df_merge.head()

,state,index,GENE_ID,score,tf
0,0,0,Rv2626c,1.0,Rv1985c
1,0,0,Rv2626c,1.0,Rv3133c
2,0,1,Rv1813c,1.0,Rv1985c
3,0,2,Rv0022c,1.0,Rv1033c
4,0,2,Rv0022c,1.0,Rv3597c


In [13]:
df_merge.to_csv("df_merge.csv")